In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.metrics import classification_report
from models import create_fixed_input_shape_model, create_variable_input_shape_model

In [ ]:
IMAGE_SIZE = (160, 160)
NUM_CLASSES = 10

test_dir = "datasets/imagenette2/val"
model_dir = "models"

In [ ]:
fixed_model = create_fixed_input_shape_model(IMAGE_SIZE, NUM_CLASSES)
fixed_model.load_weights(os.path.join(model_dir, "fixed-imagenette2.h5"))

In [ ]:
variable_model = create_variable_input_shape_model(NUM_CLASSES)
variable_model.load_weights(os.path.join(model_dir, "variable-imagenette2.h5"))

In [ ]:
CLASSES = {
    0: 'n01440764',
    1: 'n02102040',
    2: 'n02979186',
    3: 'n03000684',
    4: 'n03028079',
    5: 'n03394916',
    6: 'n03417042',
    7: 'n03425413',
    8: 'n03445777',
    9: 'n03888257'
}

In [ ]:
def predict(path, model, image_size=None, classes=CLASSES):
    image = load_img(path, color_mode="rgb", target_size=image_size)
    image = img_to_array(image)
    image = preprocess_input(image)
    image = np.expand_dims(image, axis=0)

    y_pred = model.predict(image)
    predicted = classes.get(y_pred.argmax())

    return predicted, y_pred.max()

In [ ]:
test_df = pd.DataFrame(columns=["image", "class"])
for class_ in CLASSES.values():
    images = [os.path.join(test_dir, class_, image) for image in os.listdir(os.path.join(test_dir, class_))]
    class_df = pd.DataFrame(data={"image": images, "class": class_})
    test_df = test_df.append(class_df, ignore_index=True)

In [ ]:
test_df['fixed_pred'] = np.nan
test_df['fixed_conf'] = np.nan
test_df['variable_pred'] = np.nan
test_df['variable_conf'] = np.nan

for idx in test_df.index:
    row = test_df.loc[idx]
    fixed_pred, fixed_conf = predict(row['image'], model=fixed_model, image_size=IMAGE_SIZE)
    variable_pred, variable_conf = predict(row['image'], model=variable_model, image_size=None)
    
    row['fixed_pred'] = fixed_pred
    row['fixed_conf'] = fixed_conf
    row['variable_pred'] = variable_pred
    row['variable_conf'] = variable_conf

    test_df.loc[idx] = row

In [ ]:
print(classification_report(test_df['class'], test_df['fixed_pred'], digits=4))

In [ ]:
print(classification_report(test_df['class'], test_df['variable_pred'], digits=4))